## 데이터 전처리

### 서울시 스타벅스 매장분석 (cont'd)

 - '11.데이터 수집-서울시 스타벅스 매장분석'에서 수집한 데이터를 사용해서 앞서 세운 2가지 가설을 검증한다.
     - 서울시 스타벅스 매장 목록 데이터, 
     - 인구 수 데이터, 
     - 사업체 수 데이터
     
- 서울시 스타벅스 매장 수, 인구수, 사업체 수를 시군구를 기준으로 분석한다.

- 두개 이상의 데이터를 병합하기 위해서는 공통된 기준값이 필요하다.
  여기서는 시군구명을 기준으로 데이터를 병합한다.

#### 서울시 스타벅스 매장목록, 인구, 사업체 데이터에 시군구명, 시군구코드추가
 - 스타벅스 매장목록의 주소에서 시군구명 정보를 추출한다.

In [1]:
import pandas as pd
seoul_sb = pd.read_excel('../data3/seoul_sb_list.xlsx', header=0)
seoul_sb.head()

,매장명,lat,long,주소,매장타입,전화번호
0,역삼아레나빌딩,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동),1522-3232,general
1,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232,general
2,신사역성일빌딩,37.514132,127.020563,서울특별시 강남구 강남대로 584 (논현동),1522-3232,general
3,국기원사거리,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,general
4,스탈릿대치R,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,reserve


In [2]:
seoul_sb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   매장명     545 non-null    object 
 1   lat     545 non-null    float64
 2   long    545 non-null    float64
 3   주소      545 non-null    object 
 4   매장타입    545 non-null    object 
 5   전화번호    545 non-null    object 
dtypes: float64(2), object(4)
memory usage: 25.7+ KB


In [4]:
#서울시 스타벅스 매장 목록 데이터에는 시군구명 column이 따로 없음. 
#주소에서 추출애햐 한다.
sgg_name=[]
for address in seoul_sb['주소']:
    sgg = address.split()[1]
    sgg_name.append(sgg)
    
seoul_sb['시군구명'] = sgg_name
seoul_sb.head()

,매장명,lat,long,주소,매장타입,전화번호,시군구명
0,역삼아레나빌딩,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동),1522-3232,general,강남구
1,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232,general,강남구
2,신사역성일빌딩,37.514132,127.020563,서울특별시 강남구 강남대로 584 (논현동),1522-3232,general,강남구
3,국기원사거리,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,general,강남구
4,스탈릿대치R,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,reserve,강남구


In [8]:
seoul_sb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   매장명     545 non-null    object 
 1   lat     545 non-null    float64
 2   long    545 non-null    float64
 3   주소      545 non-null    object 
 4   매장타입    545 non-null    object 
 5   전화번호    545 non-null    object 
 6   시군구명    545 non-null    object 
dtypes: float64(2), object(5)
memory usage: 29.9+ KB


In [9]:
#excel에 저장한다.
seoul_sb.to_excel('../data3/seoul_starbucks_list.xlsx', index=False)

#### 수집한 데이터 병합을 통한 분석 데이터프레임만들기
- 시군구목록 데이터에 시군구별 스타벅스 매장 수, 주민인구, 사업체 수 통계 데이터를
  병합한다.

In [5]:
seoul_sgg = pd.read_excel('../data3/seoul_sgg_list.xlsx')
seoul_sgg.head(6)

,시군구코드,시군구명,위도,경도
0,11320,도봉구,37.665861,127.031767
1,11380,은평구,37.617612,126.922700
2,11230,동대문구,37.583801,127.050700
3,11590,동작구,37.496504,126.944307
4,11545,금천구,37.460097,126.900155
5,11530,구로구,37.495486,126.858121


In [6]:
seoul_starbucks = pd.read_excel('../data3/seoul_starbucks_list.xlsx')
seoul_starbucks.columns

Index(['매장명', 'lat', 'long', '주소', '매장타입', '전화번호', '시군구명'], dtype='object')

In [7]:
#seoul_sgg로 받은 데이터 프레임의 column이름과 동일하게
#seoul_starbuck로 받은 데이터 프레임의 column명도'위도','경도'로 바꾼다
seoul_starbucks=seoul_starbucks.rename(columns={'lat':'위도','long':'경도'})

In [8]:
#실수로 매장타입, 전화번호 column 두개가 서로 바꿔져 저장되었었다. 바꾸자.
seoul_starbucks=seoul_starbucks.rename(columns={'매장타입':'전화번호','전화번호':'매장타입'})

In [9]:
seoul_starbucks.head()

,매장명,위도,경도,주소,전화번호,매장타입,시군구명
0,역삼아레나빌딩,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동),1522-3232,general,강남구
1,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232,general,강남구
2,신사역성일빌딩,37.514132,127.020563,서울특별시 강남구 강남대로 584 (논현동),1522-3232,general,강남구
3,국기원사거리,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,general,강남구
4,스탈릿대치R,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,reserve,강남구


### 추가내용: pivot연습
- seoul_starbucks 데이터프레임에서 매장타입별 count수를 구해서 
- 추후 각 시구군구별 어떤 타입의 매장이 많이 분포되어있는지 확인해보자.(추후 데이터 시각화에서 매장타입을 구분해서 매장 분포를 그리는 map으로도 확인할 수 있음)

In [10]:
seoul_starbucks.매장타입.unique()

array(['general', 'reserve', 'generalDT', 'generalWT'], dtype=object)

In [11]:
seoul_starbucks_pivot = seoul_starbucks.pivot_table(values='매장명',
                                                   index='시군구명',
                                                   columns='매장타입',
                                                   aggfunc='count')

In [12]:
seoul_starbucks_pivot

매장타입,general,generalDT,generalWT,reserve
시군구명,,,,
강남구,74.0,NaN,NaN,10.0
강동구,13.0,2.0,NaN,1.0
강북구,5.0,NaN,NaN,NaN
강서구,16.0,2.0,NaN,1.0
관악구,7.0,2.0,NaN,2.0
광진구,15.0,1.0,NaN,NaN
구로구,11.0,NaN,NaN,NaN
금천구,11.0,NaN,NaN,NaN
노원구,11.0,1.0,NaN,NaN


### dataframes 통합

시군구목록 데이터에 시군구별 스타벅스 매장 수, 주민인구, 사업체 수 통계 데이터를 병합한다.

In [16]:
import pandas as pd
seoul_sgg = pd.read_excel('../data3/seoul_sgg_list.xlsx')
print(seoul_sgg.info())
seoul_sgg.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시군구코드   25 non-null     int64  
 1   시군구명    25 non-null     object 
 2   위도      25 non-null     float64
 3   경도      25 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 928.0+ bytes
None


,시군구코드,시군구명,위도,경도
0,11320,도봉구,37.665861,127.031767
1,11380,은평구,37.617612,126.922700
2,11230,동대문구,37.583801,127.050700
3,11590,동작구,37.496504,126.944307
4,11545,금천구,37.460097,126.900155


In [17]:
starbucks_sgg_count = seoul_starbucks.pivot_table(values='매장명', 
                                                  index = '시군구명',
                                                  aggfunc='count').rename(columns={'매장명':'스타벅스_매장수'})
starbucks_sgg_count.head()

,스타벅스_매장수
시군구명,
강남구,84
강동구,16
강북구,5
강서구,19
관악구,11


서울시 시군구 목록 데이터에 스타벅스 매장 수 데이터 병합
- how='left' 시군구목록데이터를 기준으로 병합
- on = '시군구명' 서로 매칭되는 값의 컬럼명이 '시군구명' 이다.

In [18]:
seoul_sgg = pd.merge(seoul_sgg, starbucks_sgg_count, how='left', on='시군구명')
seoul_sgg.head()

,시군구코드,시군구명,위도,경도,스타벅스_매장수
0,11320,도봉구,37.665861,127.031767,2
1,11380,은평구,37.617612,126.922700,8
2,11230,동대문구,37.583801,127.050700,9
3,11590,동작구,37.496504,126.944307,11
4,11545,금천구,37.460097,126.900155,11


시군구별 주민인구, 사업체 통계 데이터를 불러와 병합한다.

In [19]:
seoul_sgg_pop = pd.read_excel('../data3/sgg_pop.xlsx')
print(seoul_sgg_pop.info())
seoul_sgg_pop.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시군구명    25 non-null     object
 1   주민증록인구  25 non-null     object
dtypes: object(2)
memory usage: 528.0+ bytes
None


,시군구명,주민증록인구
0,종로구,"161,869"
1,중구,"136,488"
2,용산구,"245,185"
3,성동구,"308,979"
4,광진구,"366,972"


In [20]:
seoul_sgg = pd.merge(seoul_sgg, seoul_sgg_pop, how='left', on='시군구명')
seoul_sgg.head()

,시군구코드,시군구명,위도,경도,스타벅스_매장수,주민증록인구
0,11320,도봉구,37.665861,127.031767,2,"335,631"
1,11380,은평구,37.617612,126.922700,8,"484,546"
2,11230,동대문구,37.583801,127.050700,9,"363,023"
3,11590,동작구,37.496504,126.944307,11,"408,912"
4,11545,금천구,37.460097,126.900155,11,"251,820"


서울시 사업체 수 통계 데이터를 병합한다.

In [22]:
seoul_sgg_biz = pd.read_excel('../data3/sgg_biz.xlsx')
print(seoul_sgg_biz.info())
seoul_sgg_biz.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시군구명    25 non-null     object
 1   사업체수    25 non-null     object
 2   종사자수    25 non-null     object
dtypes: object(3)
memory usage: 728.0+ bytes
None


,시군구명,사업체수,종사자수
0,종로구,"39,679","260,446"
1,중구,"60,127","392,568"
2,용산구,"20,254","135,240"
3,성동구,"28,343","178,243"
4,광진구,"24,445","126,305"


In [23]:
seoul_sgg = pd.merge(seoul_sgg, seoul_sgg_biz, how='left', on='시군구명')
seoul_sgg.head()

,시군구코드,시군구명,위도,경도,스타벅스_매장수,주민증록인구,사업체수,종사자수
0,11320,도봉구,37.665861,127.031767,2,"335,631","18,628","72,939"
1,11380,은평구,37.617612,126.922700,8,"484,546","24,681","93,631"
2,11230,동대문구,37.583801,127.050700,9,"363,023","31,324","138,400"
3,11590,동작구,37.496504,126.944307,11,"408,912","19,793","106,159"
4,11545,금천구,37.460097,126.900155,11,"251,820","33,814","242,686"


병합된 결과 데이터를 엑셀로 저장한다.

In [20]:
seoul_sgg.to_excel('../data3/seoul_sb_final.xlsx', index=False)